In [1]:
from tqdm.auto import trange, tqdm
import pandas as pd
import ast
import itertools
import torch
import wandb
import evaluate
from itertools import cycle
import numpy as np
import random
import time
from datetime import datetime
import collections
from sklearn.metrics import top_k_accuracy_score

In [2]:
import transformers
from transformers import get_scheduler
from transformers import AutoTokenizer
from transformers.models.bert.modeling_bert import BertModel


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch_geometric as pyg
import torch_geometric.transforms as T
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GATConv, GATv2Conv, TransformerConv, PDNConv, global_mean_pool, global_max_pool

In [4]:
transformers.__version__, pyg.__version__

('4.26.0', '2.2.0')

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

# definition

In [6]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

In [7]:
from dataclasses import dataclass
@dataclass
class myGNNoutput:
    loss: None
    logit: None
    emb: None

In [8]:
# English specific denpendency relations: https://universaldependencies.org/en/dep/
s = '''nsubj 	csubj
↳nsubj:pass 	↳csubj:pass
↳nsubj:outer 	↳csubj:outer
obj 	ccomp 	xcomp
iobj
obl 	advcl 	advmod
↳obl:npmod 	↳advcl:relcl
↳obl:tmod
vocative 	aux 	mark
discourse 	↳aux:pass
expl 	cop
nummod 	acl 	amod
  	↳acl:relcl
appos 	  	det
  	  	↳det:predet
nmod 	  	 
↳nmod:npmod
↳nmod:tmod
↳nmod:poss
compound 	flat
↳compound:prt 	↳flat:foreign
fixed 	goeswith
conj 	cc
  	↳cc:preconj
list 	parataxis 	orphan
dislocated 		reparandum
root 	punct 	dep'''
all_relations = []
s = s.split('\n')
for line in s:
    if '↳' in line:
        continue
    line = line.split('\t')
    for r in line:
        if r.strip() == '':
            continue
        all_relations.append(r.split(':')[0].strip())
if 'root' in all_relations:
    all_relations.remove('root')
    all_relations.append('ROOT')
    all_relations.append('case')      # manually add relation not in list
    all_relations.append('discourse')    # manually add relation not in list
all_relations = sorted(all_relations)

In [9]:
relation2id = {all_relations[i]:i for i in range(len(all_relations))}
relation2id['self'] = 36 # add self loop type

In [10]:
def get_loader(df, add_syllables=False, col='pos_seqs', limit=None, batch_size=32, shuffle=True, max_length=128):
    data_list = []
    if limit is not None:
        dfnew = df.sample(frac=1).reset_index(drop=True)[:limit]
    else:
        dfnew = df
    data_list = []
    count = 0
    for i in trange(len(dfnew), leave=False):
        curr = df.iloc[i]
        data = Data()
        data.edge_index = torch.cat([torch.tensor([[0],[0]]),  # for self loop of CLS token
                                     torch.tensor(curr['edge_indexs']).T, 
                                     # for batching purpose, if data.x is missing, edge_index is used to inference batch
                                     # an isolated node (the SEP in this case) will mess all up
                                     torch.tensor([[len(curr['edge_indexs'])+1],[len(curr['edge_indexs'])+1]])], 
                                    axis=1)
        data.edge_type_ids = torch.tensor([36]+[relation2id[t.split(':')[0]] for t in curr['hetoro_edges']]+[36])
        if data.edge_index.shape[1] >= max_length-1:
            count += 1
#             print(f"data {i} too long length {data.edge_index.shape[1]}")
            continue
        
        data.text = ' '.join(curr[col])
        data.y = torch.tensor([curr['author']])
        if add_syllables:
            data.num_syllables = torch.tensor([17]+curr['num_syllables']+[17])
            
        if 'doc_id' in curr:
            data.doc_id = torch.tensor([curr['doc_id']])

        data.num_nodes = len(data.edge_type_ids)
        data_list.append(data)
    print(f'{count} data dropped because of exceeding max_length {max_length}')
    loader = DataLoader(data_list, batch_size=batch_size, shuffle=shuffle)
    return loader


In [11]:
def preprocess_author_ids(df):
    assert 'author' in df, 'no column named "author" found in df'
    
    max_id, min_id = df['author'].max(), df['author'].min()
    mapping = {i+min_id:i for i in range(max_id-min_id+1)}
    df['author'] = df['author'].map(mapping)
    
    return df

In [ ]:
class mygnnblock(torch.nn.Module):
    def __init__(self):
        super().__init__()
        

In [17]:
GNNtype2layer = {'GATConv':GATConv, 'GATv2Conv':GATv2Conv, 'TransformerConv':TransformerConv, 'PDNConv':PDNConv}

class myHeteroGNN(torch.nn.Module):
    def __init__(self, 
                 num_layers, 
                 num_classes, 
                 num_dep_type, 
                 heads, 
                 hidden_dim, 
                 dep_emb_dim=32, 
                 add_self_loops=False, 
                 gnntype='GATConv', 
                 add_syllables=None,
                 checkpoint='/scratch/data_jz17d/result/pos_mlm_corenlp/retrained_256_pos_mlm_0_recovered/',
                 max_length=256,
                 dropout=0.1):
        
        super().__init__()
        self.checkpoint = checkpoint
        self.max_length = max_length
        self.tokenizer = AutoTokenizer.from_pretrained(self.checkpoint, local_files_only=True)
        self.bert = BertModel.from_pretrained(self.checkpoint, local_files_only=True, add_pooling_layer = False).to(device)
        self.num_layers = num_layers
        self.num_classes = num_classes
        self.pos_emb_dim = 32 # this is determined by POS Bert
        self.heads = heads
        self.hidden_dim = hidden_dim  # self.pos_emb_dim//self.heads
        self.dep_emb_dim = dep_emb_dim
        self.add_syllables = add_syllables
        self.dropout = dropout
        
        if add_syllables:
            self.num_syllables = 18 # the longest word has 17 syllables
            self.syllable_emb_layer = nn.Embedding(self.num_syllables, self.pos_emb_dim)
            
        self.GNNlayer = GNNtype2layer[gnntype]
        
        self.add_self_loops = add_self_loops
        self.dep_emb_layer = nn.Embedding(num_dep_type, self.dep_emb_dim)
        
        self.gnns = nn.ModuleList()
        self.gnns.append(self.GNNlayer(self.pos_emb_dim, self.hidden_dim, heads = self.heads, add_self_loops=self.add_self_loops, edge_dim=self.dep_emb_dim, beta=True))
        for i in range(self.num_layers-1):
            self.gnns.append(self.GNNlayer(self.hidden_dim * self.heads, self.hidden_dim, heads = self.heads, edge_dim=self.dep_emb_dim, beta=True))
        
        self.layernorms = nn.ModuleList()
        self.layernorms.append(nn.LayerNorm(self.pos_emb_dim))
        for i in range(self.num_layers-1):
            self.layernorms.append(nn.LayerNorm(self.hidden_dim * self.heads))
        
        self.classifier = nn.Linear(2*self.hidden_dim * self.heads, self.num_classes)
        self.lossfn = nn.CrossEntropyLoss()
        
    def forward(self, text, edge_index, edge_type_ids, batch, y, ptr, num_syllable=None, readout='pool'):
        tokens = self.tokenizer(text, padding=True, truncation=True, max_length=self.max_length, return_tensors='pt').to(device)
        x = self.bert(**tokens).last_hidden_state
        # reshape! drop padded tokens!
        x = x.masked_select(tokens.attention_mask.ge(0.5).unsqueeze(2)).reshape((-1,self.pos_emb_dim))
        
        if self.add_syllables:
            syllable_emb = self.syllable_emb_layer(num_syllable)
            x = x + syllable_emb
            
        edge_attr = self.dep_emb_layer(edge_type_ids)
        for i in range(self.num_layers):
            if i == 0:
                x = self.gnns[i](self.layernorms[i](x), edge_index, edge_attr=edge_attr).relu()
            else:
                x = self.gnns[i](self.layernorms[i](x), edge_index, edge_attr=edge_attr).relu() + x
            x = F.dropout(x, p=self.dropout, training=self.training)
                
        if readout == 'pool':
            x = torch.cat([global_mean_pool(x, batch), global_max_pool(x, batch)], axis=1)
        elif readout == 'cls':
            x = x[ptr[:-1],:]
        
        x = F.dropout(x, p=self.dropout, training=self.training)
        logit = self.classifier(x)
        loss = self.lossfn(logit, y)
        return myGNNoutput(loss=loss, logit=logit, emb=x)

In [18]:
cols_to_eval = ['edge_indexs', 'hetoro_edges', 'pos_seqs', 'upos_seqs', 'num_syllables', 'alignments']

# train

In [ ]:
max_length = 256

epochs = 100
warmup_ratio = 0.15

LIMIT = [None]
NUM_LAYERS = [4]
LR = [1e-3]
HEADS = [4]
READOUT = ['pool']
GNNTYPE = ['TransformerConv'] # 'GATConv', 'GATv2Conv', 
HIDDEN_DIM = [32]
DEP_EMB_DIM = [32]
NUM_SENT = [1,2,3]
ADD_SELF_LOOPS = [False]
ADD_SYLLABLES = [True]
DROPOUT = [0.1]
REPEAT = list(range(1))

ARGS = itertools.product(LIMIT, NUM_LAYERS, LR, HEADS, READOUT, GNNTYPE, HIDDEN_DIM, DEP_EMB_DIM, NUM_SENT, ADD_SELF_LOOPS, ADD_SYLLABLES, DROPOUT, REPEAT)
num_runs = len(list(ARGS))
run_pbar = trange(num_runs, leave=False)

skip_runs = -1
ARGS = itertools.product(LIMIT, NUM_LAYERS, LR, HEADS, READOUT, GNNTYPE, HIDDEN_DIM, DEP_EMB_DIM, NUM_SENT, ADD_SELF_LOOPS, ADD_SYLLABLES, DROPOUT, REPEAT)
for i_run, args in enumerate(ARGS):

    if i_run <= skip_runs:
        run_pbar.update(1)
        continue
    limit, num_layers, lr, heads, readout, gnntype, hidden_dim, dep_emb_dim, num_sent_per_text, add_self_loops, add_syllables, dropout, repeat = args
    
    seed = int(datetime.now().timestamp())
    set_seed(seed)
    
    file = f'../../data/CCAT50/processed/author_all_sent_{num_sent_per_text}_train.csv'
    df = pd.read_csv(file)
    for col in cols_to_eval:
        df[col] = df[col].apply(ast.literal_eval)

    file = f'../../data/CCAT50/processed/author_all_sent_{num_sent_per_text}_val.csv'
    df_val = pd.read_csv(file)
    for col in cols_to_eval:
        df_val[col] = df_val[col].apply(ast.literal_eval)
    val_docid2index = {doc_id:i for i,doc_id in enumerate(df_val['doc_id'].unique())}
    
    valid_loader = get_loader(df_val, add_syllables=add_syllables, max_length=max_length)
    num_valid_steps = len(valid_loader)
    train_loader = get_loader(df, limit = limit, add_syllables=add_syllables, max_length=max_length)
    num_training_steps = len(train_loader)
    
    model = myHeteroGNN(num_layers=num_layers,
                       num_classes=50, 
                       num_dep_type=len(relation2id), 
                       heads=heads,
                       hidden_dim=hidden_dim,
                       dep_emb_dim=dep_emb_dim, 
                       add_self_loops=add_self_loops,
                       gnntype=gnntype,
                       add_syllables=add_syllables,
                       dropout=dropout,
                      )
    
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    scheduler = get_scheduler("linear",
                            optimizer=optimizer,
                            num_warmup_steps=int(warmup_ratio*epochs*num_training_steps),
                            num_training_steps=epochs*num_training_steps)
    
    wconfig = {}
    wconfig['seed'] = seed
    wconfig['num_sent_per_text'] = num_sent_per_text
    wconfig['limit'] = limit
    wconfig['num_layers'] = num_layers
    wconfig['lr'] = lr
    wconfig['heads'] = heads
    wconfig['readout'] = readout
    wconfig['GNNtype'] = gnntype
    wconfig['add_self_loops'] = add_self_loops
    wconfig['add_syllables'] = add_syllables
    wconfig['dropout'] = dropout
    
    run = wandb.init(project="hetero POS GNN (layernorm and dropout)", 
                     entity="fsu-dsc-cil", 
                     dir='/scratch/data_jz17d/wandb_tmp/', 
                     config=wconfig,
                     name=f'run_{i_run}',
                     reinit=True,
                     settings=wandb.Settings(start_method='thread'))
    
    best_evaluation = collections.defaultdict(float)
    pbar = trange(epochs*num_training_steps, leave=False)
    for i_epoch in range(epochs):
        model.train()
        for data in train_loader:
            data.to(device)
            optimizer.zero_grad()
            if add_syllables:
                output = model(data.text, data.edge_index, data.edge_type_ids, data.batch, data.y, data.ptr, data.num_syllables, readout=readout)
            else:
                output = model(data.text, data.edge_index, data.edge_type_ids, data.batch, data.y, data.ptr, readout=readout)
            output.loss.backward()
            optimizer.step()
            scheduler.step()
            pbar.update(1)

        model.eval()
        doc_score = 1e-8*np.ones((len(val_docid2index),50))
        doc_true = np.zeros(len(val_docid2index))
        metric = evaluate.load('/home/jz17d/Desktop/metrics/accuracy')
        for data in valid_loader:
            data.to(device)
            if add_syllables:
                output = model(data.text, data.edge_index, data.edge_type_ids, data.batch, data.y, data.ptr, data.num_syllables, readout=readout)
            else:
                output = model(data.text, data.edge_index, data.edge_type_ids, data.batch, data.y, data.ptr, readout=readout)
            metric.add_batch(predictions=output.logit.argmax(axis=-1).cpu().detach().numpy(), references=data.y.cpu().numpy())
            
            pred = output.logit.argmax(axis=-1).cpu().detach().numpy()
            doc_id = np.vectorize(val_docid2index.get)(data.doc_id.cpu().detach().numpy()) 
            doc_score[doc_id,pred] += 1
            doc_true[doc_id] = data.y.cpu().numpy()
        
        # logging
        evaluation = metric.compute()
        for k in range(1, 6):
            evaluation.update({f'doc_acc@{k}': top_k_accuracy_score(doc_true, doc_score, k=k)})
        wandb.log(evaluation, step=pbar.n)
        
        # logging best
        for key in evaluation:
            best_evaluation[f'best_{key}'] = max(best_evaluation[f'best_{key}'], evaluation[key])
        wandb.log(best_evaluation, step=pbar.n)
    
    run.finish()
    run_pbar.update(1)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/8207 [00:00<?, ?it/s]

0 data dropped because of exceeding max_length 256


  0%|          | 0/32937 [00:00<?, ?it/s]

1 data dropped because of exceeding max_length 256


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/103000 [00:00<?, ?it/s]

# sweep

In [ ]:
sweep_configuration = {
    'method': 'bayes',
    'name': 'sweep',
    'metric': {
        'goal': 'maximize', 
        'name': 'best_doc_acc@1'
        },
    'parameters': {
        'num_layers': {'distribution': 'int_uniform', 'min': 1, 'max': 8},
        'heads': {'distribution': 'int_uniform', 'min': 1, 'max': 8},
        'hidden_dim': {'values': [16, 32, 64]},
        'dep_emb_dim': {'values': [16, 32, 64]},
        'epochs': {'values': [100, 150, 200]},
        'lr': {'min': 0.0001, 'max': 0.01}
     },
    'early_terminate':{
        'type': 'hyperband',
        'max_iter': 100000,
        'eta': 5,
        's': 4,
    }
}

In [ ]:
def run_sweep():
    
    run = wandb.init(dir='/scratch/data_jz17d/wandb_tmp/')
    
    limit = None
    num_layers = wandb.config.num_layers
    heads = wandb.config.heads
    readout = 'pool'
    gnntype = 'TransformerConv'
    hidden_dim = wandb.config.hidden_dim
    dep_emb_dim = wandb.config.dep_emb_dim
    epochs = wandb.config.epochs
    lr = wandb.config.lr
    
    num_sent_per_text = 1
    add_self_loops = False
    add_syllables = True
    max_length = 256
    warmup_ratio = 0.15
    num_classes = 50
    
    seed = int(datetime.now().timestamp())
    set_seed(seed)
    
    file = f'../../data/CCAT50/processed/author_all_sent_{num_sent_per_text}_train.csv'
    df = pd.read_csv(file)
    for col in cols_to_eval:
        df[col] = df[col].apply(ast.literal_eval)

    file = f'../../data/CCAT50/processed/author_all_sent_{num_sent_per_text}_val.csv'
    df_val = pd.read_csv(file)
    for col in cols_to_eval:
        df_val[col] = df_val[col].apply(ast.literal_eval)
    val_docid2index = {doc_id:i for i,doc_id in enumerate(df_val['doc_id'].unique())}
    
    valid_loader = get_loader(df_val, add_syllables=add_syllables, max_length=max_length)
    num_valid_steps = len(valid_loader)
    train_loader = get_loader(df, limit = limit, add_syllables=add_syllables, max_length=max_length)
    num_training_steps = len(train_loader)
    
    model = myHeteroGNN(num_layers=num_layers,
                       num_classes=num_classes, 
                       num_dep_type=len(relation2id), 
                       heads=heads,
                       hidden_dim=hidden_dim,
                       dep_emb_dim=dep_emb_dim, 
                       add_self_loops=add_self_loops,
                       gnntype=gnntype,
                       add_syllables=add_syllables
                      )
    
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    scheduler = get_scheduler("linear",
                            optimizer=optimizer,
                            num_warmup_steps=int(warmup_ratio*epochs*num_training_steps),
                            num_training_steps=epochs*num_training_steps)
    
    
    best_evaluation = collections.defaultdict(float)
    pbar = trange(epochs*num_training_steps, leave=False)
    for i_epoch in range(epochs):
        model.train()
        for data in train_loader:
            data.to(device)
            optimizer.zero_grad()
            if add_syllables:
                output = model(data.text, data.edge_index, data.edge_type_ids, data.batch, data.y, data.ptr, data.num_syllables, readout=readout)
            else:
                output = model(data.text, data.edge_index, data.edge_type_ids, data.batch, data.y, data.ptr, readout=readout)
            output.loss.backward()
            optimizer.step()
            scheduler.step()
            pbar.update(1)

        model.eval()
        doc_score = 1e-8*np.ones((len(val_docid2index), num_classes))
        doc_true = np.zeros(len(val_docid2index))
        metric = evaluate.load('/home/jz17d/Desktop/metrics/accuracy')
        for data in valid_loader:
            data.to(device)
            if add_syllables:
                output = model(data.text, data.edge_index, data.edge_type_ids, data.batch, data.y, data.ptr, data.num_syllables, readout=readout)
            else:
                output = model(data.text, data.edge_index, data.edge_type_ids, data.batch, data.y, data.ptr, readout=readout)
            metric.add_batch(predictions=output.logit.argmax(axis=-1).cpu().detach().numpy(), references=data.y.cpu().numpy())
            
            pred = output.logit.argmax(axis=-1).cpu().detach().numpy()
            doc_id = np.vectorize(val_docid2index.get)(data.doc_id.cpu().detach().numpy()) 
            doc_score[doc_id,pred] += 1
            doc_true[doc_id] = data.y.cpu().numpy()
        
        # logging
        evaluation = metric.compute()
        for k in range(1, 6):
            evaluation.update({f'doc_acc@{k}': top_k_accuracy_score(doc_true, doc_score, k=k)})
        wandb.log(evaluation, step=pbar.n)
        
        # logging best
        for key in evaluation:
            best_evaluation[f'best_{key}'] = max(best_evaluation[f'best_{key}'], evaluation[key])
        wandb.log(best_evaluation, step=pbar.n)
    


In [ ]:
sweep_id = wandb.sweep(sweep=sweep_configuration, project='hetero-gnn-sweep')
wandb.agent(sweep_id, function=run_sweep)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: kuy0bogu
Sweep URL: https://wandb.ai/fsu-dsc-cil/hetero-gnn-sweep/sweeps/kuy0bogu


wandb: Agent Starting Run: ic82g576 with config:
wandb: 	dep_emb_dim: 32
wandb: 	epochs: 150
wandb: 	heads: 4
wandb: 	hidden_dim: 64
wandb: 	lr: 0.003387887451680067
wandb: 	num_layers: 1
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: cpuyyp (fsu-dsc-cil). Use `wandb login --relogin` to force relogin


  0%|          | 0/8207 [00:00<?, ?it/s]

0 data dropped because of exceeding max_length 256


...

wandb: ERROR Abnormal program exit
Run 24j09uo3 errored: Exception('problem')
wandb: ERROR Run 24j09uo3 errored: Exception('problem')
wandb: Agent Starting Run: p78xtm34 with config:
wandb: 	dep_emb_dim: 64
wandb: 	epochs: 200
wandb: 	heads: 4
wandb: 	hidden_dim: 64
wandb: 	lr: 0.0010174435846466367
wandb: 	num_layers: 1
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
Traceback (most recent call last):
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/wandb/sdk/wandb_init.py", line 1073, in init
    wi.setup(kwargs)
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/wandb/sdk/wandb_init.py", line 165, in setup
    self._wl = wandb_setup.setup()
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/wandb/sdk/wandb_setup.py", line 312, in setup
    ret = _setup(settings=settings)
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packag

wandb: ERROR Abnormal program exit
Run mc4kneno errored: Exception('problem')
wandb: ERROR Run mc4kneno errored: Exception('problem')
wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: wuitxkym with config:
wandb: 	dep_emb_dim: 64
wandb: 	epochs: 200
wandb: 	heads: 6
wandb: 	hidden_dim: 64
wandb: 	lr: 0.0015326513325644425
wandb: 	num_layers: 3
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
Traceback (most recent call last):
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/wandb/sdk/wandb_init.py", line 1073, in init
    wi.setup(kwargs)
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/wandb/sdk/wandb_init.py", line 165, in setup
    self._wl = wandb_setup.setup()
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/wandb/sdk/wandb_setup.py", line 312, in setup
    ret = _setup(settings=settings)
  File "

wandb: ERROR Abnormal program exit
Run k3wiu5wt errored: Exception('problem')
wandb: ERROR Run k3wiu5wt errored: Exception('problem')
wandb: Agent Starting Run: 1a2iwmq1 with config:
wandb: 	dep_emb_dim: 64
wandb: 	epochs: 100
wandb: 	heads: 8
wandb: 	hidden_dim: 64
wandb: 	lr: 0.0060365126040084266
wandb: 	num_layers: 8
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
Traceback (most recent call last):
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/wandb/sdk/wandb_init.py", line 1073, in init
    wi.setup(kwargs)
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/wandb/sdk/wandb_init.py", line 165, in setup
    self._wl = wandb_setup.setup()
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/wandb/sdk/wandb_setup.py", line 312, in setup
    ret = _setup(settings=settings)
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packag

wandb: ERROR Abnormal program exit
Run ttclae89 errored: Exception('problem')
wandb: ERROR Run ttclae89 errored: Exception('problem')
wandb: Agent Starting Run: 0s4klmsw with config:
wandb: 	dep_emb_dim: 32
wandb: 	epochs: 200
wandb: 	heads: 7
wandb: 	hidden_dim: 64
wandb: 	lr: 0.003273699567983906
wandb: 	num_layers: 5
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
Traceback (most recent call last):
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/wandb/sdk/wandb_init.py", line 1073, in init
    wi.setup(kwargs)
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/wandb/sdk/wandb_init.py", line 165, in setup
    self._wl = wandb_setup.setup()
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/wandb/sdk/wandb_setup.py", line 312, in setup
    ret = _setup(settings=settings)
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-package

wandb: ERROR Abnormal program exit
Run ux3w9w0r errored: Exception('problem')
IOStream.flush timed out
wandb: ERROR Run ux3w9w0r errored: Exception('problem')
wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 4f0i5w59 with config:
wandb: 	dep_emb_dim: 32
wandb: 	epochs: 200
wandb: 	heads: 6
wandb: 	hidden_dim: 16
wandb: 	lr: 0.0057721913910097
wandb: 	num_layers: 7
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
Traceback (most recent call last):
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/wandb/sdk/wandb_init.py", line 1073, in init
    wi.setup(kwargs)
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/wandb/sdk/wandb_init.py", line 165, in setup
    self._wl = wandb_setup.setup()
  File "/home/jz17d/anaconda3/envs/torch/lib/python3.9/site-packages/wandb/sdk/wandb_setup.py", line 312, in setup
    ret = _setup(setti